# Pulling Tweets from WHO

In [10]:
# Import libbraries
import os, sys, subprocess
import json
import pandas as pd
import numpy as np
from langdetect import detect
import pickle

pd.set_option('display.max_colwidth', -1)

pd.set_option('display.max_rows', 1000)
#pd.set_option('display.max_columns', 500)

In [11]:
# load project config

terminal_call = ! git rev-parse --show-toplevel
repo_path=terminal_call[0]
project_config_path = os.path.join(repo_path,'project_config.json')

with open(project_config_path,'r') as fp: 
    project_config = json.load(fp)


project_config

{'project_module_relative_path': 'src'}

In [12]:
# import custom scraper module

module_path = os.path.join(repo_path,project_config['project_module_relative_path'])
sys.path.append(module_path)

import scraper
from scraper import tweet_scraper

from importlib import reload

## Pull Tweets for a year

In [18]:
# get tweets from for a year from this date
df = tweet_scraper.pull_tweets('AstraZeneca', '2019-06-01', '2020-06-01')

In [19]:
# rename columns
df.rename(columns={0:"date",
          1:"text"}, inplace = True)

In [20]:
# set column types
df['text'] = df['text'].astype(str)
df['date'] = pd.to_datetime(df['date'])

## Pickle for later use

In [29]:
pickle.dump(df, open("../data/who_tweets.pkl", "wb"))

## Clean Tweets

In [23]:
# Reload package when I make updates to function
# reload(tweet_scraper)

In [21]:
# Clean tweet column
df = tweet_scraper.clean_tweets(df, 'text')

# Write to CSV

In [22]:
# write to csv
df.to_csv("../data/astra_tweets_clean.csv", index = False)